In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix
import anndata

In [2]:
moudata = anndata.read_h5ad('../2/3 clustered_mou_0831_anno.h5ad')
humdata = anndata.read_h5ad('../2/3 clustered_hum_0831_anno.h5ad')

moudata_nn = sc.read_h5ad('2 filtered_normalised_scaled_mou_0902.h5ad') # not regressed
humdata_nn = sc.read_h5ad('2 filtered_normalised_scaled_hum_0902.h5ad')

/home/lh3n19/.conda/envs/my/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
np.sum(humdata_nn.X<0)

0

In [5]:
mou_fm = pd.read_pickle('4 mou_featuremap_0829')
hum_fm = pd.read_pickle('4 hum_featuremap_0829')

mou_fm['Paralogue'] = mou_fm['ix_DGE_mm'].duplicated() | mou_fm['ix_DGE_hs'].duplicated()
hum_fm['Paralogue'] = hum_fm['ix_DGE_mm'].duplicated() | hum_fm['ix_DGE_hs'].duplicated()

mgiSymbol_hsVarGenes = mou_fm[mou_fm['hsapiens_homolog_associated_gene_name'].isin(humdata.var_names[humdata.var['highly_variable']])]['mgi_symbol']

genesToConsider = list(set(mgiSymbol_hsVarGenes).union(set(moudata.var_names[moudata.var['highly_variable']])))

In [6]:
len(genesToConsider)

5067

In [9]:
mou_fm['UseForTrainingNN'] = moudata.var_names[mou_fm['ix_DGE_mm'].astype('int32')].isin(genesToConsider)
FeatureSubset = ~(mou_fm['mgi_symbol'].duplicated() | mou_fm['hsapiens_homolog_associated_gene_name'].duplicated()) & mou_fm['UseForTrainingNN']

In [10]:
mou_fm

,mgi_symbol,ensembl_gene_id,hsapiens_homolog_ensembl_gene,hsapiens_homolog_associated_gene_name,ix_DGE_mm,ix_DGE_hs,Paralogue,UseForTrainingNN
4,0610010F05Rik,ENSMUSG00000042208,ENSG00000162929,KIAA1841,8.0,9405.0,False,True
5,0610010K14Rik,ENSMUSG00000020831,ENSG00000258315,C17orf49,9.0,2768.0,False,False
6,0610030E20Rik,ENSMUSG00000058706,ENSG00000168887,C2orf68,12.0,2953.0,False,True
9,0610040J01Rik,ENSMUSG00000060512,ENSG00000154274,C4orf19,17.0,2990.0,False,False
14,1110004F10Rik,ENSMUSG00000030663,ENSG00000110696,C11orf58,26.0,2663.0,False,False
...,...,...,...,...,...,...,...,...
15956,Zxdc,ENSMUSG00000034430,ENSG00000070476,ZXDC,16567.0,23696.0,False,True
15957,Zyg11b,ENSMUSG00000034636,ENSG00000162378,ZYG11B,16568.0,23698.0,False,False
15958,Zyx,ENSMUSG00000029860,ENSG00000159840,ZYX,16569.0,23699.0,False,True
15959,Zzef1,ENSMUSG00000055670,ENSG00000074755,ZZEF1,16570.0,23700.0,False,False


In [19]:
# centered
x_mouse = moudata.X[:,mou_fm['ix_DGE_mm'].astype('int32')]
x_mouse = x_mouse[:, FeatureSubset]
x_mou_col = mou_fm[FeatureSubset]['mgi_symbol']
x_mou_esb = mou_fm[FeatureSubset]['ensembl_gene_id']
y_mou = moudata.obs['CellLabel'].to_numpy()
np.savez_compressed('5 x_mou_0831_anno.npz', x = x_mouse, col = x_mou_col, esb = x_mou_esb, y = y_mou)

x_hum = humdata.X[:,mou_fm['ix_DGE_hs'].astype('int32')]
x_hum = x_hum[:, FeatureSubset]
y_hum = humdata.obs['CellLabel'].to_numpy()
np.savez_compressed('5 x_hum_0831_anno.npz', x = x_hum, y = y_hum)

In [12]:
# non negative
x_mouse = moudata_nn.X[:,mou_fm['ix_DGE_mm'].astype('int32')]
x_mouse = x_mouse[:, FeatureSubset]
#x_mou_col = mou_fm[FeatureSubset]['mgi_symbol']
#x_mou_esb = mou_fm[FeatureSubset]['ensembl_gene_id']
y_mou = moudata.obs['CellLabel'].to_numpy()
np.savez_compressed('5 x_mou_0902_anno_nn.npz', x = x_mouse, y = y_mou)

x_hum = humdata_nn.X[:,mou_fm['ix_DGE_hs'].astype('int32')]
x_hum = x_hum[:, FeatureSubset]
y_hum = humdata.obs['CellLabel'].to_numpy()
np.savez_compressed('5 x_hum_0902_anno_nn.npz', x = x_hum, y = y_hum)

In [15]:
x_mouse

<5512x4313 sparse matrix of type '<class 'numpy.float32'>'
	with 1125777 stored elements in Compressed Sparse Row format>